In [ ]:
from datasets import load_dataset
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import clip

In [ ]:
dataset = load_dataset("starvector/text2svg-stack")

In [ ]:
train_df_all = dataset["train"].to_pandas()
test_df_all = dataset["test"].to_pandas()

In [ ]:
print(len(train_df_all))
print(len(test_df_all))

2169710
5709


In [ ]:
def preprocess_svg_df(df, top_n):
    # width, height 필터링
    def extract_svg_dims(svg_str):
        match = re.search(r'<svg[^>]*width="(\d+)"[^>]*height="(\d+)"', svg_str)
        if match:
            return int(match.group(1)), int(match.group(2))
        return None, None

    df["svg_dims"] = df["Svg"].apply(extract_svg_dims)
    df_filtered = df[df["svg_dims"].apply(
        lambda dims: dims[0] is not None and 800 <= dims[0] <= 1000 and 800 <= dims[1] <= 1000
    )].copy()

    # SVG 길이 기준 상위 N개 선택
    df_filtered["svg_length"] = df_filtered["Svg"].apply(len)
    df_sorted = df_filtered.sort_values("svg_length", ascending=False).head(top_n).copy()

    # 텍스트/SVG 추출
    input_texts = df_sorted["caption_blip2"].astype(str).tolist() #caption_blip2
    target_svgs = df_sorted["Svg"].astype(str).tolist()

    return input_texts, target_svgs, df_sorted

In [ ]:
input_train, target_train, df_train = preprocess_svg_df(train_df_all, top_n = 1000)
input_test, target_test, df_test = preprocess_svg_df(test_df_all, top_n = 300)

In [ ]:
print(len(df_train))
print(len(df_test))

1000
11


양자화(Quantization)

LLaMA 2 7B는 엄청 큰 모델이라 일반적인 GPU (예: 12GB~24GB VRAM)에서는 **풀 정밀도(float32)**나 **반 정밀도(float16)**로는 로드조차 안됨

4bit 양자화를 통해서 : 모델 사이즈를 크게 줄이고 (4배 이상 작게) VRAM 사용량도 줄임 / 성능은 최대한 유지

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "meta-llama/Llama-2-7b-chat-hf"
hf_token = ""

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_4bit_enable_fp32_cpu_offload=True  # CPU 오프로드 허용
)


tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # GPU에 자동 분산 로딩
    quantization_config=quant_config,
    token=hf_token
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PEFT(Parameter-Efficient Fine-Tuning) : 모델 전체를 다 학습하지 않고, 일부만 학습해서 성능을 유지하면서도 학습 비용을 낮추는 방법 / 기존 LLM은 수십억 개 파라미터 → 전체 파라미터 학습은 매우 비쌈 / PEFT는 일부 파라미터만 학습해서 성능은 비슷하게, 자원은 적게 -> 대표적인 PEFT 방식 중 하나가 바로 LoRA


LoRA(Low-Rank Adaptation) : 기존 모델 파라미터를 고정한 채,"작은 추가 레이어만 학습"해서 fine-tuning 하는 방법

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"▶ Trainable / Total = {trainable:,} / {total:,}")

▶ Trainable / Total = 4,194,304 / 3,504,607,232


In [ ]:
from datasets import Dataset

# Hugging Face Dataset 구성
train_dataset = Dataset.from_dict({
    "input": input_train,
    "output": target_train
})

# 프롬프트 포맷
def format_chat_prompt(example):
    return f"""Given the <CAPTION>, generate the corresponding svg code.
Return a response WITH SVG CODE that appropriately alignes with the request, IT SHOULD BE VERY ELABORATE.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

Please ensure that the generated SVG code is well-formed, valid, and strictly adheres to these constraints.
Focus on a clear and concise representation of the input description within the given limitations. Always give the complete SVG code with nothing omitted. Never use an ellipsis.

<constraints>
* **Allowed Elements:** `svg`, `path`, `circle`, `rect`, `ellipse`, `line`, `polyline`, `polygon`, `g`, `linearGradient`, `radialGradient`, `stop`, `defs`
* **Allowed Attributes:** `viewBox`, `width`, `height`, `fill`, `stroke`, `stroke-width`, `d`, `cx`, `cy`, `r`, `x`, `y`, `rx`, `ry`, `x1`, `y1`, `x2`, `y2`, `points`, `transform`, `opacity`
</constraints>

<CAPTION>
{example['input']}
</CAPTION>

SVG:{example['output']}"""


#prompt :

train_dataset = train_dataset.map(lambda x: {"text": format_chat_prompt(x)})

# 토크나이즈
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = train_dataset.map(tokenize, batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# 1) 레이블 마스킹 함수 정의
import copy, torch

def add_labels(example):
    input_ids = example["input_ids"]
    # 텐서면 clone, 리스트면 copy
    labels = input_ids.clone() if isinstance(input_ids, torch.Tensor) else copy.deepcopy(input_ids)

    # "SVG:" 토큰 아이디 찾기
    svg_ids = tokenizer("SVG:", add_special_tokens=False).input_ids
    svg_id  = svg_ids[-1] if svg_ids else None
    try:
        seq      = labels.tolist() if isinstance(labels, torch.Tensor) else labels
        start_idx = seq.index(svg_id)
    except ValueError:
        start_idx = -1

    if start_idx >= 0:
        # prompt 부분을 -100으로 마스킹
        if isinstance(labels, torch.Tensor):
            labels[: start_idx+1] = -100
        else:
            labels[: start_idx+1] = [-100] * (start_idx+1)

    example["labels"] = labels
    return example

# 2) map → set_format
tokenized_dataset = tokenized_dataset.map(add_labels, batched=False)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# 3) DataLoader 생성 (이제 labels가 포함됩니다)
from torch.utils.data import DataLoader
from transformers import default_data_collator
dataloader = DataLoader(
    tokenized_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=default_data_collator
)

# 4) Optimizer
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=1e-4)

# 5) 학습 루프
device = torch.device("cuda")
model.to(device).train()

for epoch in range(5):
    total_loss = 0
    for step, batch in enumerate(dataloader):
        optimizer.zero_grad()
        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels         = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if step % 50 == 0:
            print(f"Epoch {epoch+1}, Step {step}, Loss: {loss.item():.4f}")

    print(f"Epoch {epoch+1} Completed. Avg Loss: {total_loss/len(dataloader):.4f}")


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Epoch 1, Step 0, Loss: 1.4752
Epoch 1, Step 50, Loss: 0.2732


In [ ]:
from huggingface_hub import login
login(token = hf_token)  # 너의 Hugging Face access token 입력


from peft import PeftModel

# 업로드할 이름 (repo name) 설정
repo_name = "llama2-text2svg-lora"

# Hugging Face Hub에 업로드 (LoRA 모델만 저장됨)
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jiseong99/llama2-text2svg-lora/commit/dfe139503b32669c767453e2a8dbd1fb9d026805', commit_message='Upload tokenizer', commit_description='', oid='dfe139503b32669c767453e2a8dbd1fb9d026805', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jiseong99/llama2-text2svg-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='jiseong99/llama2-text2svg-lora'), pr_revision=None, pr_num=None)

In [ ]:
from huggingface_hub import hf_hub_download
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import huggingface_hub

# 4bit 양자화 설정
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True,
)

# Hugging Face repo
adapter_repo = "jiseong99/llama2-text2svg-lora"

# ❗ 사전 다운로드 - timeout 설정 추가 (기본 10초 → 60초)
hf_hub_download(repo_id=adapter_repo, filename="adapter_model.safetensors")

# LoRA config
peft_config = PeftConfig.from_pretrained(adapter_repo)

# ✅ Base model 강제 로딩
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    torch_dtype=torch.float16,
    quantization_config=quant_config,
    low_cpu_mem_usage=False,
    device_map=None
)

# ✅ LoRA adapter 적용
model = PeftModel.from_pretrained(
    base_model,
    adapter_repo,
    device_map="auto"
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(adapter_repo)
tokenizer.pad_token = tokenizer.eos_token


adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

In [ ]:
import cairosvg
from PIL import Image
import io

In [ ]:
import clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
def generate_svg(caption):
    model.eval()

    prompt = f"""Given the <CAPTION>, generate the corresponding svg code.
Return a response WITH SVG CODE that appropriately alignes with the request, IT SHOULD BE VERY ELABORATE.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

Please ensure that the generated SVG code is well-formed, valid, and strictly adheres to these constraints.
Focus on a clear and concise representation of the input description within the given limitations. Always give the complete SVG code with nothing omitted. Never use an ellipsis.

<constraints>
* **Allowed Elements:** `svg`, `path`, `circle`, `rect`, `ellipse`, `line`, `polyline`, `polygon`, `g`, `linearGradient`, `radialGradient`, `stop`, `defs`
* **Allowed Attributes:** `viewBox`, `width`, `height`, `fill`, `stroke`, `stroke-width`, `d`, `cx`, `cy`, `r`, `x`, `y`, `rx`, `ry`, `x1`, `y1`, `x2`, `y2`, `points`, `transform`, `opacity`
</constraints>

<CAPTION>
{caption}
</CAPTION>

SVG:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=4096,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            top_k=50
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded


In [ ]:
def svg_to_image(svg_code):
    png_bytes = cairosvg.svg2png(bytestring=svg_code.encode("utf-8"))
    image = Image.open(io.BytesIO(png_bytes)).convert("RGB")
    return image

In [ ]:
def compute_clip_score(caption, svg_code):
    if not svg_code.strip():
        return 0.0

    try:
        image = svg_to_image(svg_code)
    except Exception as e:
        print(f"❌ SVG to image failed: {e}")
        return 0.0

    image_input = preprocess(image).unsqueeze(0).to(device)
    text_input = clip.tokenize([caption]).to(device)

    with torch.no_grad():
        image_features = clip_model.encode_image(image_input)
        text_features = clip_model.encode_text(text_input)

        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        similarity = (image_features @ text_features.T).item()
        return similarity

In [ ]:
def extract_svg_from_output(output: str) -> str:
    # "SVG:" 이후 부분을 추출
    if "SVG:" in output:
        output = output.split("SVG:")[-1].strip()
    # 혹시 이상한 HTML/텍스트가 껴도 <svg부터 자름
    svg_start = output.find("<svg")
    if svg_start != -1:
        return output[svg_start:].strip()
    return ""  # fallback


In [ ]:
df_train.tail()

,Filename,Svg,caption_blip2,caption_cogvlm,caption_llava,svg_dims,svg_length
1522066,2581a4e5bc55d0a423b8259282440d1061cf6d46.svg,"<svg xmlns=""http://www.w3.org/2000/svg"" width=...",a cartoon ghost with a big smile on its face,"The image showcases a simple, hand-drawn ghost...",A cartoon ghost with a surprised expression on...,"(1000, 1000)",7591
220861,3756609ca86f8474e31e78afae391ac57dd2d205.svg,"<svg version=""1.2"" baseProfile=""tiny-ps"" xmlns...","the logo for the company, which is called dg",The image showcases a geometric design compose...,A blue and white logo with a blue arrow pointi...,"(808, 808)",7588
2059532,e632d4d9f91d34ac86d52db534d3516ace1b4285.svg,"<svg\n xmlns:dc=""http://purl.org/dc/elements...",a black and white square with the number 1 on it,The image appears to be a black and white grap...,A black square with a white number one in the ...,"(1000, 1000)",7584
318101,f3ebdd5ba1d4207d0c876f666fff299915be84b5.svg,"<svg\n width=""1000""\n height=""1000""\n vi...",chinese calligraphy font for the word love,The image showcases a black-colored Chinese ch...,The image features a large black character or ...,"(1000, 1000)",7580
1466895,a1bde490db4e7b6f49a09e5b6da30f7d6f4f925d.svg,"<svg\n xmlns:dc=""http://purl.org/dc/elements...",red bird flying over water vector,"The image showcases a silhouette of a bird, po...","The image features a red bird, possibly a duck...","(1000, 1000)",7573


In [ ]:
generated_svg = generate_svg("a cartoon ghost with a big smile on its face") #black and white checker pants

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


In [ ]:
extracted_svg = extract_svg_from_output(generated_svg)

In [ ]:
print(generated_svg)

Given the <CAPTION>, generate the corresponding svg code.
Return a response WITH SVG CODE that appropriately alignes with the request, IT SHOULD BE VERY ELABORATE.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

Please ensure that the generated SVG code is well-formed, valid, and strictly adheres to these constraints.
Focus on a clear and concise representation of the input description within the given limitations. Always give the complete SVG code with nothing omitted. Never use an ellipsis.

<constraints>
* **Allowed Elements:** `svg`, `path`, `circle`, `rect`, `ellipse`, `line`, `polyline`, `polygon`, `g`, `linearGradient`, `radialGradient`, `stop`, `defs`
* **Allowed Attributes:** `viewBox`, `width`, `height`, `fill`, `stroke`, `stroke-width`, `d`, `cx`, `cy`, `r`, `x`, `y`, `rx`, `ry`, `x1`, `y1`, `x2`, `y2`, `points`, `transform`, `opacity`
</constraints>

<CAPTION>
a cartoon ghost with a

In [ ]:
print(extracted_svg)

<svg viewBox="0 0 100 100" xmlns="http://www.w3.org/2000/svg"><g fill="#fff" fill-rule="non-zero"><path d="M 10 20 C 20 20 30 40 50 60 C 60 70 70 80 80 90 C 90 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 100 100 100 C 100 100 100 

In [ ]:
'''
from tqdm import tqdm

created_svgs = []
clip_scores = []

for caption in tqdm(df_test["caption_blip2"], desc="Generating SVGs"):
    # 1. SVG 생성
    raw_output = generate_svg(caption)

    # 2. 정제
    svg_code = extract_svg_from_output(raw_output)
    created_svgs.append(svg_code)

    # 3. CLIP score 계산
    score = compute_clip_score(caption, svg_code)
    clip_scores.append(score)

# 🔹 컬럼 추가
df_test["created_svg"] = created_svgs
df_test["clip_score"] = clip_scores

# 🔹 평균 CLIPScore 출력
average_score = sum(clip_scores) / len(clip_scores)
print(f"\n✅ CLIPScore Average: {average_score:.4f}")
'''

'\nfrom tqdm import tqdm\n\ncreated_svgs = []\nclip_scores = []\n\nfor caption in tqdm(df_test["caption_blip2"], desc="Generating SVGs"):\n    # 1. SVG 생성\n    raw_output = generate_svg(caption)\n\n    # 2. 정제\n    svg_code = extract_svg_from_output(raw_output)\n    created_svgs.append(svg_code)\n\n    # 3. CLIP score 계산\n    score = compute_clip_score(caption, svg_code)\n    clip_scores.append(score)\n\n# 🔹 컬럼 추가\ndf_test["created_svg"] = created_svgs\ndf_test["clip_score"] = clip_scores\n\n# 🔹 평균 CLIPScore 출력\naverage_score = sum(clip_scores) / len(clip_scores)\nprint(f"\n✅ CLIPScore Average: {average_score:.4f}")\n'

In [ ]:
df_train